In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb

# Directories
for d in ["raw_data"]:
    if not os.path.isdir(d):
        os.mkdir(d)

In [2]:
# Upload trips data
hogar_21 = pd.read_excel(f"raw_data/survey_2021.xlsx", sheet_name = 'HOGAR')
individuo_21 = pd.read_excel(f"raw_data/survey_2021.xlsx", sheet_name = 'INDIVIDUO')
trips_21 = pd.read_excel(f"raw_data/survey_2021.xlsx", sheet_name = 'DESPLAZAMIENTOS')

hogar_16 = pd.read_excel(f"raw_data/survey_2016.xlsx", sheet_name = 'HOGAR')
individuo_16 = pd.read_excel(f"raw_data/survey_2016.xlsx", sheet_name = 'INDIVIDUO')
trips_16 = pd.read_excel(f"raw_data/survey_2016.xlsx", sheet_name = 'DESPLAZAMIENTOS')

hogar_11 = pd.read_excel(f"raw_data/survey_2011.xlsx", sheet_name = 'E_Hogares')
individuo_11 = pd.read_excel(f"raw_data/survey_2011.xlsx", sheet_name = 'E_Personas')
trips_11 = pd.read_excel(f"raw_data/survey_2011.xlsx", sheet_name = 'E_Desplazamientos')

# Fix differences between years: "HOGAR", "INDIVIDUOS" and "DESPLAZAMIENTOS"

"HOGAR"

In [3]:
# In 2011 towns are categorized as 01 002 instead of 01002.

hogar_11['Municipio'] = hogar_11['Municipio'].apply(lambda x: "".join(x.split()))
hogar_11['Municipio'] = pd.to_numeric(hogar_11['Municipio'], errors='coerce', downcast='integer')

# In 2016, capitals are not inlcuded in regions

hogar_16['Comarca'] = hogar_16['Comarca'].replace({21: 2, 22: 8, 23: 17})

# In 2011 there are two regions that are not in Euskadi. Nevertheless, there is not any household there.

"PERSONAS"

In [4]:
# In 2016 and 2011 female sex is 6 instead of 2.
individuo_16['Sexo'] = individuo_16['Sexo'].replace({6: 2})
individuo_11['Sexo'] = individuo_11['Sexo'].replace({6: 2})

# In 2021 there in an extra category: 0-6 years. There are individuals in that category, but 0 displacaments.

# In 2016 and 2011, NOT having a license is 6 instead of 2.

individuo_16['crnt_tur'] = individuo_16['crnt_tur'].replace({6: 2})
individuo_11['crnt_tur'] = individuo_11['crnt_tur'].replace({6: 2})

individuo_16['crnt_mot'] = individuo_16['crnt_mot'].replace({6: 2})
individuo_11['crnt_mot'] = individuo_11['crnt_mot'].replace({6: 2})

individuo_16['crnt_otr'] = individuo_16['crnt_otr'].replace({6: 2})
individuo_11['crnt_otr'] = individuo_11['crnt_otr'].replace({6: 2})

# The NO displacements is 6 instead of 2

individuo_16['desplaz'] = individuo_16['desplaz'].replace({6: 2})
individuo_11['desplaz'] = individuo_11['desplaz'].replace({6: 2})

# There are some individuals that do not make any displacement, or shorter than 5 min or doesn't apply.

"DESPLAZAMIENTOS"

In [5]:
# 2021

# Drop rows that have NaN un Com_Ori of 1021
trips_21 = trips_21.dropna(subset=['Com_Ori'])

# Change them into int
trips_21['Com_Ori'] = trips_21['Com_Ori'].astype(int)
trips_21['Mun_Ori'] = trips_21['Mun_Ori'].astype(int)

# There are 4 rowns with a 7 that we don't know what they are. Delete.
trips_21 = trips_21[trips_21['Peaje'] != 7]

In [6]:
# 2016

# In 2016 and 2011 there is a blank space in Bill and Park that means that doesn't apply.
trips_16['Bill'] = trips_16['Bill'].replace({' ': 9})
trips_16['Park'] = trips_16['Park'].replace({' ': 9})
trips_16['Peaje'] = trips_16['Peaje'].replace({' ': 9})

# 2011
trips_11['Bill'].fillna(9, inplace=True)
trips_11['Park'].fillna(9, inplace=True)
trips_11['Peaje'].fillna(9, inplace=True)

trips_11['Bill'] = trips_11['Bill'].astype(int)
trips_11['Park'] = trips_11['Park'].astype(int)

In [7]:
# Comarcas origin and destination of 2016

trips_16['Com_Ori'] = trips_16['Com_Ori'].replace({21: 2, 22: 8, 23: 17})
trips_16['Com_Dest'] = trips_16['Com_Dest'].replace({21: 2, 22: 8, 23: 17})

# Town of origin and destination of 2011
trips_11['Mun_Ori'] = trips_11['Mun_Ori'].apply(lambda x: "".join(x.split()))
trips_11['Mun_Des'] = trips_11['Mun_Des'].apply(lambda x: "".join(x.split()))
trips_11['Mun_Ori'] = pd.to_numeric(trips_11['Mun_Ori'], errors='coerce', downcast='integer')
trips_11['Mun_Des'] = pd.to_numeric(trips_11['Mun_Des'], errors='coerce', downcast='integer')

# In 2011 there is not activity type 12.

trips_11['Act_Ori'] = trips_11['Act_Ori'].replace({12: 13})
trips_11['Act_Des'] = trips_11['Act_Des'].replace({12: 13})

# In 2016 "Otros" is with P7 instead of P6 LIKE IN 2021. I first delete P6 (NC).
trips_16 = trips_16[trips_16['Cod_Prop'] != "P6"]
trips_16['Cod_Prop'] = trips_16['Cod_Prop'].replace({"P7": "P6"})

# Change codes of 2011.

trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({1: "P5"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({2: "P2"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({3: "P1"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({4: "P1"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({5: "P3"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({6: "P3"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({7: "P4"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({8: "P6"})
trips_11['Cod_Prop'] = trips_11['Cod_Prop'].replace({9: "P6"})

# There is a unknown mode of transportation called FEVE in 2011. It's in Bizkaia so I delete it.
trips_16 = trips_16[trips_16['Cod_Modo'] != 22]
trips_11['Cod_Modo'] = trips_11['Cod_Modo'].replace({23: 22, 24: 23, 25: 24})

# Different code of parking in 2016 and 2011
trips_16['Park'] = trips_16['Park'].replace({3: 2})
trips_16['Park'] = trips_16['Park'].replace({5: 3})
trips_16['Park'] = trips_16['Park'].replace({6: 4})
trips_16['Park'] = trips_16['Park'].replace({7: 5})

trips_11['Park'] = trips_11['Park'].replace({3: 2})
trips_11['Park'] = trips_11['Park'].replace({5: 3})
trips_11['Park'] = trips_11['Park'].replace({6: 4})
trips_11['Park'] = trips_11['Park'].replace({7: 5})

# Toll is with 6 and not 2.
trips_16['Peaje'] = trips_16['Peaje'].replace({6: 2})

# In 2016 blank space means that it doesnt apply.
trips_16['Peaje'] = trips_16['Peaje'].replace({0: 1})

trips_11['Peaje'] = trips_11['Peaje'].replace({'SI': 2})
trips_11['Peaje'] = trips_11['Peaje'].replace({'NO': 1})

# Merge HOGAR Y PERSONA, to add HH type

In [8]:
people_11 = pd.merge(individuo_11, hogar_11, on='ID_Hogar', how='left')
people_16 = pd.merge(individuo_16, hogar_16, on='ID_Hogar', how='left')
people_21 = pd.merge(individuo_21, hogar_21, on='ID_Hogar', how='left')

people_11['Tipo_familia'] = ''
people_16['Tipo_familia'] = ''
people_21['Tipo_familia'] = ''

def anadir_tipo_familia(df, lista_ID_Hogar):
  valor_estudiante = [4]
  for id in lista_ID_Hogar:
    m = df[df['ID_Hogar'] == id]
    per_hog = m["Per_hog"].iloc[0]
    conteo = m['Actividad'].isin(valor_estudiante).sum()
    if len(m) == conteo: # all students
      df.loc[m.index, 'Tipo_familia'] = 'All students'
    if per_hog == 1:
      if len(m) == 1:
        df.loc[m.index, 'Tipo_familia'] = 'Hogar de una persona'
      else:
        conteo = m['Actividad'].isin(valor_estudiante).sum() # counts number of students
        if conteo == 0:
          df.loc[m.index, 'Tipo_familia'] = '2 adultos'
        else:
          df.loc[m.index, 'Tipo_familia'] = '1 adulto con niño(s)'
    elif per_hog == 2:
      if len(m) == 3:
        conteo = m['Actividad'].isin(valor_estudiante).sum()
        if conteo == 2:
          df.loc[m.index, 'Tipo_familia'] = '1 adulto con niño(s)'
        elif conteo == 1:
          df.loc[m.index, 'Tipo_familia'] = '2 adultos con niño(s)'
        elif conteo == 0:
          df.loc[m.index, 'Tipo_familia'] = 'Otros hogares sin niños'
      elif len(m) == 4:
        conteo = m['Actividad'].isin(valor_estudiante).sum()
        if conteo == 3:
          df.loc[m.index, 'Tipo_familia'] = '1 adulto con niño(s)'
        elif conteo == 2:
          df.loc[m.index, 'Tipo_familia'] = '2 adultos con niño(s)'
        elif conteo == 1:
          df.loc[m.index, 'Tipo_familia'] = 'Otros hogares con niños'
        elif conteo == 0:
          df.loc[m.index, 'Tipo_familia'] = 'Otros hogares sin niños'
      elif len(m) == 5:
        conteo = m['Actividad'].isin(valor_estudiante).sum()
        if conteo == 4:
          df.loc[m.index, 'Tipo_familia'] = '1 adulto con niño(s)'
        elif conteo == 3:
          df.loc[m.index, 'Tipo_familia'] = '2 adultos con niño(s)'
        elif conteo == 0:
          df.loc[m.index, 'Tipo_familia'] = 'Otros hogares sin niños'
        else:
          df.loc[m.index, 'Tipo_familia'] = 'Otros hogares con niños'
    else: # Per_hog == 3
      conteo = m['Actividad'].isin(valor_estudiante).sum()
      if (len(m)-conteo) == 1:
        df.loc[m.index, 'Tipo_familia'] = '1 adulto con niño(s)'
      elif (len(m)-conteo) == 2:
        df.loc[m.index, 'Tipo_familia'] = '2 adultos con niño(s)'
      elif conteo == 0:
        df.loc[m.index, 'Tipo_familia'] = 'Otros hogares sin niños'
      else:
        df.loc[m.index, 'Tipo_familia'] = 'Otros hogares con niños'

anadir_tipo_familia(people_11, people_11['ID_Hogar'].unique())
anadir_tipo_familia(people_16, people_16['ID_Hogar'].unique())
anadir_tipo_familia(people_21, people_21['ID_Hogar'].unique())

In [9]:
# To  each trip, I assign its trip-taker and HH characteristics using merge

trips_21 = pd.merge(trips_21, people_21, on='ID_Pers', how='left')
trips_21 = trips_21.drop(columns='ID_Hogar_y')
trips_21 = trips_21.rename(columns={'ID_Hogar_x': 'ID_Hogar'})

trips_16 = pd.merge(trips_16, people_16, on='ID_Pers', how='left')
trips_16 = trips_16.drop(columns='ID_Hogar_y')
trips_16 = trips_16.rename(columns={'ID_Hogar_x': 'ID_Hogar'})

trips_11 = pd.merge(trips_11, people_11, on='ID_Pers', how='left')
trips_11 = trips_11.drop(columns='ID_Hogar_y')
trips_11 = trips_11.rename(columns={'ID_Hogar_x': 'ID_Hogar'})

In [10]:
# Delete trips with origin and destination out of Gipuzkoa

trips_21 = trips_21[(trips_21['Prov_Ori'] == 20) | (trips_21['Prov_Des'] == 20)]
trips_16 = trips_16[(trips_16['Prov_Ori'] == 20) | (trips_16['Prov_Des'] == 20)]
trips_11 = trips_11[(trips_11['Prov_Ori'] == 20) | (trips_11['Prov_Des'] == 20)]

# Delete trips that travel out of Spain

trips_21 = trips_21[(trips_21['Pais_Ori'] == 108) & (trips_21['Pais_Ori'] == 108)]
trips_16 = trips_16[(trips_16['Pais_Ori'] == 108) & (trips_16['Pais_Ori'] == 108)]
trips_11 = trips_11[(trips_11['Pais_Ori'] == 108) & (trips_11['Pais_Ori'] == 108)]

# Delete unnecessesary columns

trips_21.rename(columns={'N_viaje': 'N_Viaje'}, inplace=True)

trips_21 = trips_21.drop(columns=['Pais_Ori', 'Pais_Des', 'desplaz', 'ID_Desp', 'N_Viaje'])
trips_16 = trips_16.drop(columns=['Pais_Ori', 'Pais_Des', 'desplaz', 'ID_Desp', 'N_Viaje'])
trips_11 = trips_11.drop(columns=['Pais_Ori', 'Pais_Des', 'desplaz', 'ID_Desp', 'N_Viaje'])

# Reset indexes

trips_21 = trips_21.reset_index()
trips_21 = trips_21.drop(columns='index')
trips_16 = trips_16.reset_index()
trips_16 = trips_16.drop(columns='index')
trips_11 = trips_11.reset_index()
trips_11 = trips_11.drop(columns='index')

trips_11 = trips_11.drop(columns=['O_TP', 'TP_D'])

In [11]:
trips_11['Recur'] = trips_11['Recur'].replace({0: 99})

def mapear_valor(valor):
    if 1 <= valor <= 2:
        return 21
    elif 3 <= valor <= 5:
        return 22
    elif 6 <= valor <= 10:
        return 23
    elif 11 <= valor <= 15:
        return 24
    elif valor != 99:
      return 25
    else:
        return valor  

trips_11['Recur'] = trips_11['Recur'].apply(mapear_valor)

In [12]:
# Concat three datasets

trips_21['Año'] = 2021
trips_16['Año'] = 2016
trips_11['Año'] = 2011

trips = pd.concat([trips_21, trips_16, trips_11], ignore_index=True)
trips

,ID_Hogar,ID_Pers,Prov_Ori,Com_Ori,Mun_Ori,Act_Ori,Prov_Des,Com_Dest,Mun_Des,Act_Des,...,crnt_mot,crnt_otr,Provincia,Municipio,Comarca,Per_hog,Turismos,Motos,Tipo_familia,Año
0,8,19,20,17,20069,2,20,17,20069,5,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
1,8,19,20,17,20069,5,20,17,20069,8,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
2,8,19,20,17,20069,8,20,17,20069,2,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
3,8,19,1,2,1999,1,20,17,20069,2,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
4,20,47,20,16,20999,5,1,2,1999,1,...,2,2,1,1999,2,2,4,1,2 adultos con niño(s),2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30614,41164,411641,20,18,20999,3,20,18,20999,1,...,2,2,20,20999,18,1,2,1,Hogar de una persona,2011
30615,41898,418981,20,18,20080,1,20,18,20080,3,...,2,2,20,20080,18,1,2,1,2 adultos,2011
30616,41898,418981,20,18,20080,3,20,18,20080,1,...,2,2,20,20080,18,1,2,1,2 adultos,2011
30617,41898,418982,20,18,20080,1,20,18,20080,7,...,2,2,20,20080,18,1,2,1,2 adultos,2011


In [13]:
trips = trips.dropna(subset=['Com_Ori'])

In [14]:
# Change float values to int values

trips['Com_Ori'] = trips['Com_Ori'].apply(lambda x: int(x) if isinstance(x, float) else x)
trips['Mun_Ori'] = trips['Mun_Ori'].apply(lambda x: int(x) if isinstance(x, float) else x)
trips['Dur_Tot'] = trips['Dur_Tot'].apply(lambda x: int(x) if isinstance(x, float) else x)
trips['elev'] = trips['elev'].apply(lambda x: int(x) if isinstance(x, float) else x)
trips = trips.reset_index()
trips = trips.drop(columns='index')
trips

,ID_Hogar,ID_Pers,Prov_Ori,Com_Ori,Mun_Ori,Act_Ori,Prov_Des,Com_Dest,Mun_Des,Act_Des,...,crnt_mot,crnt_otr,Provincia,Municipio,Comarca,Per_hog,Turismos,Motos,Tipo_familia,Año
0,8,19,20,17,20069,2,20,17,20069,5,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
1,8,19,20,17,20069,5,20,17,20069,8,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
2,8,19,20,17,20069,8,20,17,20069,2,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
3,8,19,1,2,1999,1,20,17,20069,2,...,2,2,1,1999,2,2,2,1,2 adultos con niño(s),2021
4,20,47,20,16,20999,5,1,2,1999,1,...,2,2,1,1999,2,2,4,1,2 adultos con niño(s),2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30614,41164,411641,20,18,20999,3,20,18,20999,1,...,2,2,20,20999,18,1,2,1,Hogar de una persona,2011
30615,41898,418981,20,18,20080,1,20,18,20080,3,...,2,2,20,20080,18,1,2,1,2 adultos,2011
30616,41898,418981,20,18,20080,3,20,18,20080,1,...,2,2,20,20080,18,1,2,1,2 adultos,2011
30617,41898,418982,20,18,20080,1,20,18,20080,7,...,2,2,20,20080,18,1,2,1,2 adultos,2011


## Assign names to interesting variables
Prov_Ori y Prov_Des
Com_Ori y Com_Dest
Mun_Ori y Mun_Des
Cod_Modo

In [15]:
# Bring the codes
comarcas = pd.read_excel(f"raw_data/surveys_codes.xlsx", sheet_name = 'Comarcas')
municipios = pd.read_excel(f"raw_data/surveys_codes.xlsx", sheet_name = 'Municipios')
actividades = pd.read_excel(f"raw_data/surveys_codes.xlsx", sheet_name = 'Actividades')
propositos = pd.read_excel(f"raw_data/surveys_codes.xlsx", sheet_name = 'Propositos')
modos = pd.read_excel(f"raw_data/surveys_codes.xlsx", sheet_name = 'Modos')

In [16]:
# Comarcas

trips = pd.merge(trips, comarcas, left_on='Comarca', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Comarca'])
trips = trips.rename(columns={'Comarcas': 'Comarca'})

trips = pd.merge(trips, comarcas, left_on='Com_Ori', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Com_Ori'])
trips = trips.rename(columns={'Comarcas': 'Com_Ori'})

trips = pd.merge(trips, comarcas, left_on='Com_Dest', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Com_Dest'])
trips = trips.rename(columns={'Comarcas': 'Com_Des'})

# Municipios

trips = pd.merge(trips, municipios, left_on='Municipio', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Municipio'])
trips = trips.rename(columns={'Municipios': 'Municipio'})

trips = pd.merge(trips, municipios, left_on='Mun_Ori', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Mun_Ori'])
trips = trips.rename(columns={'Municipios': 'Mun_Ori'})

trips = pd.merge(trips, municipios, left_on='Mun_Des', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Mun_Des'])
trips = trips.rename(columns={'Municipios': 'Mun_Des'})

# Actividades

trips = pd.merge(trips, actividades, left_on='Act_Ori', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Act_Ori'])
trips = trips.rename(columns={'Actividades': 'Act_Ori'})

trips = pd.merge(trips, actividades, left_on='Act_Des', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Act_Des'])
trips = trips.rename(columns={'Actividades': 'Act_Des'})

# Propositos

trips = pd.merge(trips, propositos, left_on='Cod_Prop', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Cod_Prop'])
trips = trips.rename(columns={'Propositos': 'Proposito'})

# Modos

trips = pd.merge(trips, modos, left_on='Cod_Modo', right_on='Codigo', how='left')
trips = trips.drop(columns=['Codigo', 'Cod_Modo'])
trips = trips.rename(columns={'Modos': 'Modo'})


In [17]:
# Order the dataframe

trips = trips[['Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des', 'Mun_Ori', 'Mun_Des', 'Act_Ori',
        'Act_Des', 'Proposito', 'N_Etapas', 'Modo', 'Hora_Ini',
       'Dur_Tot', 'Recur', 'Bill', 'Park', 'Peaje', 'elev', 'Provincia',
       'Municipio', 'Comarca', 'Per_hog', 'Turismos', 'Motos', 'Tipo_familia', 'Sexo', 'Edad',
       'Actividad', 'crnt_tur', 'crnt_mot', 'crnt_otr', 'Año']]

In [18]:
trips = trips[trips['N_Etapas'] == 1]

eliminar = ['N_Etapas']
trips = trips.drop(columns=eliminar)
trips = trips.reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Turismos,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año
0,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
1,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
2,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
3,20,1,Debagoiena,Arabako Lautada/Llanada Alavesa,Resto Gipuzkoa,Resto Alava/Araba,Centro estudios,Residencia Habitual,Estudio,Coche conductor,...,4,1,2 adultos con niño(s),2,3,4,1,2,2,2021
4,1,20,Arabako Lautada/Llanada Alavesa,Debagoiena,Resto Alava/Araba,Resto Gipuzkoa,Residencia Habitual,Centro estudios,Estudio,Coche conductor,...,4,1,2 adultos con niño(s),2,3,4,1,2,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30309,20,20,Goierri,Goierri,Resto Gipuzkoa,Resto Gipuzkoa,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,Hogar de una persona,1,5,1,1,2,2,2011
30310,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
30311,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
30312,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,1,2 adultos,2,5,6,2,2,2,2011


In [19]:
# Trips inside Gipuzkoa, 28400 rows out of 30300.
# I do it because I can't add the time from Bilbao because I don't have the network. It could be done.

trips = trips[trips['Prov_Ori'] == 20]
trips = trips[trips['Prov_Des'] == 20]
trips = trips.reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Turismos,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año
0,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
1,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
2,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
3,20,20,Donostialdea,Goierri,Donostia/SanSebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,1,2021
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/SanSebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28392,20,20,Goierri,Goierri,Resto Gipuzkoa,Resto Gipuzkoa,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,Hogar de una persona,1,5,1,1,2,2,2011
28393,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
28394,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
28395,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,1,2 adultos,2,5,6,2,2,2,2011


In [20]:
# Delete rows that have Mun_Ori or Mun_Des "Resto de Gipuzkoa"

trips = trips[trips['Mun_Ori'] != 'Resto Gipuzkoa']
trips = trips[trips['Mun_Des'] != 'Resto Gipuzkoa']
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Turismos,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año
0,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
1,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
2,20,20,Donostialdea,Donostialdea,Donostia/SanSebastian,Donostia/SanSebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
3,20,20,Donostialdea,Goierri,Donostia/SanSebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,1,2021
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/SanSebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28386,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,2,1,Hogar de una persona,1,6,3,1,2,2,2011
28393,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
28394,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
28395,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,1,2 adultos,2,5,6,2,2,2,2011


In [21]:
trips = trips.replace("Donostia/SanSebastian", "Donostia/San Sebastian")

In [22]:
# Change on "Hora_Ini" column

trips['Hora_Ini'] = trips['Hora_Ini'].astype(str)

def convertir_a_formato_deseado(tiempo):
    partes = tiempo.split(':')
    if len(partes) == 3:
        # If seconds, convert to "HH:MM"
        return ':'.join(partes[:2])
    else:
        return tiempo

# Apply function
trips['Hora_Ini'] = trips['Hora_Ini'].apply(convertir_a_formato_deseado)

# Insert 0 before 7:20, for example. 07:20.
trips['Hora_Ini'] = trips['Hora_Ini'].astype(str).str.zfill(5)

# Modify weird rows.
trips['Hora_Ini'] = trips['Hora_Ini'].replace('1900-01-02 17:00', '17:00')
# trips = trips[trips['ID_Pers'] != 7009]
trips = trips[trips['Hora_Ini'] != '00nan']

trips = trips.reset_index(drop=True)
# trips['Hora_Ini'] = trips['Hora_Ini'].replace('00nan', '17:00')

# Define round function
def redondear_hora_abajo(hora_str):
    partes = hora_str.split(':')
    hora = int(partes[0])
    minutos = int(partes[1])

    minutos_redondeados = minutos // 5 * 5
    if minutos_redondeados == 60:
        hora += 1
        minutos_redondeados = 0

    hora_redondeada = f"{hora:02d}:{minutos_redondeados:02d}"
    return hora_redondeada

# Apply function
trips['Hora_Ini'] = trips['Hora_Ini'].apply(redondear_hora_abajo)
trips['Hora_Ini'].unique()
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Turismos,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,1,2021
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22112,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,2,1,Hogar de una persona,1,6,3,1,2,2,2011
22113,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
22114,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
22115,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,1,2 adultos,2,5,6,2,2,2,2011


In [23]:
# Delete rows with NaN o NS/NC on Mun_Ori or Mun_Des

pueblos = ['Donostia/San Sebastian', 'Arrasate/Mondragon', 'Urnieta',
       'Andoain', 'Zarautz', 'Beasain', 'Hernani', 'Irun', 'Usurbil',
       'Villabona', 'Lasarte-Oria', 'Hondarribia', 'Tolosa',
       'Aretxabaleta', 'Bergara', 'Azkoitia', 'Ordizia', 'Azpeitia',
       'Elgoibar', 'Zumaia', 'Orio', 'Zumarraga', 'Lazkao', 'Legazpi',
       'Oñati', 'Deba', 'Eibar', 'Urretxu', 'Errenteria', 'Lezo',
       'Oiartzun', 'Pasaia', 'Mutriku']

trips = trips[trips['Mun_Ori'].isin(pueblos)]
trips = trips[trips['Mun_Des'].isin(pueblos)]
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Turismos,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,1,2 adultos con niño(s),2,2,4,2,2,2,2021
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,1,2021
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,3,1,2 adultos con niño(s),1,5,1,1,2,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22112,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,2,1,Hogar de una persona,1,6,3,1,2,2,2011
22113,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
22114,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,1,2 adultos,1,5,1,1,2,2,2011
22115,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,1,2 adultos,2,5,6,2,2,2,2011


In [48]:
trips.to_csv("../input_data/trips.csv")

In [49]:
trips = pd.read_csv("../input_data/trips.csv",index_col=0)

# Add income level data

In [24]:
# Income based on munipality
# https://www.eustat.eus/elementos/ele0005700/renta-personal-media-de-la-c-a-de-euskadi-por-ambitos-territoriales-segun-tipo-de-renta-euros/tbl0005790_c.html
income_municipality = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsx", sheet_name = 'Municipio')
income_municipality

# Income based on age and gender
# https://www.eustat.eus/elementos/ele0002100/renta-personal-media-de-la-ca-de-euskadi-por-sexo-y-edad-quinquenal-segun-tipo-de-renta-euros/tbl0002166_c.html
income_age_man = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsx", sheet_name = 'Hombres_edad', usecols="D,E")
income_age_woman = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsx", sheet_name = 'Mujeres_edad', usecols="D,E")
income_age_man = income_age_man.dropna()
income_age_woman = income_age_woman.dropna()

# Income based on Actividad
# https://www.eustat.eus/elementos/ele0002100/renta-personal-media-por-sexo-y-relacion-con-la-actividad-segun-tipo-de-renta-euros/tbl0002170_c.html
# https://www.eustat.eus/elementos/ele0002900/renta-personal-media-de-poblacion-jubilada-por-tipo-de-renta-segun-territorio-historico-y-sexo-euros/tbl0002942_c.html
income_actividad = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsx", sheet_name = 'Actividad', usecols="E,F")
income_actividad = income_actividad.dropna()

# Income based on number of cars
# Deciles: https://www.eustat.eus/elementos/ele0013700/poblacionde18ymasanos-pordecilderenta-personal-de-la-ca-de-euskadi-segun-territoriohistorico-y-sexo-/tbl0013735_c.html
income_cuartiles = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsX", sheet_name = 'Cuartiles', usecols="D,E")
income_cuartiles = income_cuartiles.dropna()
income_coches = pd.read_excel(f"raw_data/income_sociodemographics/income_sociodemographics.xlsx", sheet_name = 'Probabilidad_coches', usecols="G,H,I,J,K")
income_coches = income_coches.dropna()

# Income based on family type
# https://www.ine.es/jaxiT3/Datos.htm?t=10942
# Estos datos los he cogido del INE porque en Eustat hay renta del hogar dependiendo del numero de personas en el hogar, bastante inexacta para mi. Este me gusta más.
income_family_21 = {
    'Tipo_familia': ['Hogar de una persona', '2 adultos', 'Otros hogares sin niños', '1 adulto con niño(s)', '2 adultos con niño(s)', 'Otros hogares con niños', 'All students'],
    'Inc_fam': [16660, 15626, 12917, 7626, 10079, 8935, 0]
}
income_family_21 = pd.DataFrame(income_family_21)
income_family_16 = {
    'Tipo_familia': ['Hogar de una persona', '2 adultos', 'Otros hogares sin niños', '1 adulto con niño(s)', '2 adultos con niño(s)', 'Otros hogares con niños', 'All students'],
    'Inc_fam': [15187, 13264, 10952, 8256, 8956, 7482, 0]
}
income_family_16 = pd.DataFrame(income_family_16)
income_family_11 = {
    'Tipo_familia': ['Hogar de una persona', '2 adultos', 'Otros hogares sin niños', '1 adulto con niño(s)', '2 adultos con niño(s)', 'Otros hogares con niños', 'All students'],
    'Inc_fam': [14989, 13270, 11783, 7434, 9115, 8299, 0]
}
income_family_11 = pd.DataFrame(income_family_11)

# el income de actividad=estudiante y la edad=2 me lo he inventado, pero necesitaba un valor

# Assign income

In [25]:
# Municipality
trips = pd.merge(trips, income_municipality, on='Municipio', how='left')
trips['Inc_municipio'] = trips['Inc_municipio'].fillna(23354) # Average salary on Euskadi. I have to do it because some says "Resto de Gipuzkoa"

# Age and gender
trips = pd.merge(trips, income_age_man, left_on='Edad', right_on='Edad.1', how='left')
trips = pd.merge(trips, income_age_woman, left_on='Edad', right_on='Edad.1', how='left')
trips['Inc_edad'] = np.where(trips['Sexo'] == 1, trips['Inc_edad_man'], trips['Inc_edad_woman'])
trips = trips.drop(columns=['Edad.1_x', 'Edad.1_y', 'Inc_edad_man', 'Inc_edad_woman'])

# Actividad
trips = pd.merge(trips, income_actividad, left_on='Actividad', right_on='Actividad.1', how='left')
trips = trips.drop(columns=['Actividad.1'])
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Inc_municipio,Inc_edad,Inc_act
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,2,4,2,2,2,2021,23354.0,5570.700000,5366
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,2,4,2,2,2,2021,23354.0,5570.700000,5366
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,2,4,2,2,2,2021,23354.0,5570.700000,5366
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,5,1,1,2,1,2021,23354.0,35240.333333,28495
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,5,1,1,2,2,2021,21819.0,35240.333333,28495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21899,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,1,6,3,1,2,2,2011,21757.0,32710.000000,19338
21900,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,5,1,1,2,2,2011,21626.0,35240.333333,28495
21901,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,5,1,1,2,2,2011,21626.0,35240.333333,28495
21902,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,2,5,6,2,2,2,2011,21626.0,23785.666667,5366


In [26]:
# Family type

df1 = pd.merge(trips[trips['Año'] == 2021], income_family_21, on='Tipo_familia', how='left')
df2 = pd.merge(trips[trips['Año'] == 2016], income_family_16, on='Tipo_familia', how='left')
df3 = pd.merge(trips[trips['Año'] == 2011], income_family_11, on='Tipo_familia', how='left')

# Prin result
trips = pd.concat([df1, df2, df3], ignore_index=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Inc_municipio,Inc_edad,Inc_act,Inc_fam
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,2,4,2,2,2,2021,23354.0,5570.700000,5366,10079
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,2,4,2,2,2,2021,23354.0,5570.700000,5366,10079
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,2,4,2,2,2,2021,23354.0,5570.700000,5366,10079
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,5,1,1,2,1,2021,23354.0,35240.333333,28495,10079
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,5,1,1,2,2,2021,21819.0,35240.333333,28495,10079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21899,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,6,3,1,2,2,2011,21757.0,32710.000000,19338,14989
21900,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,5,1,1,2,2,2011,21626.0,35240.333333,28495,13270
21901,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,5,1,1,2,2,2011,21626.0,35240.333333,28495,13270
21902,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,5,6,2,2,2,2011,21626.0,23785.666667,5366,13270


In [27]:
# Cars
import random
# Function to assign a column based on number of cars
def asignar_columna(row):
    numero_turismos = row['Turismos']
    probabilidades = income_coches.loc[income_coches['Num_cars'] == numero_turismos][['Q1.1', 'Q2.1', 'Q3.1', 'Q4.1']].values[0]
    columnas_posibles = ['Q1.1', 'Q2.1', 'Q3.1', 'Q4.1']
    columna_asignada = np.random.choice(columnas_posibles, p=probabilidades / sum(probabilidades))
    return columna_asignada

# Apply function
trips['Cuartil_coche'] = trips.apply(asignar_columna, axis=1)

# Assign income as a function of the cuartile
trips = pd.merge(trips, income_cuartiles, left_on='Cuartil_coche', right_on='Cuartil', how='left')
trips = trips.drop(columns=['Cuartil_coche', 'Cuartil'])

In [28]:
# Income final

trips['Income'] = trips['Inc_act']*0.45 + trips['Inc_car']*0.15 + trips['Inc_edad']*0.15 + trips['Inc_municipio']*0.1 + trips['Inc_fam']*0.15
# If any of the values of income is 0 (kid, student...), write 0 on the final income.
def update_income(row):
    if row['Inc_edad'] < 1 or row['Inc_act'] < 1 or row['Inc_fam'] < 1: # 0: people between 0-19 años. Actividad=estudiante. Inc_fam==all students.
        return 0
    else:
        return row['Income']
# Apply the function to update 'Column4'
trips['Income'] = trips.apply(update_income, axis=1)
trips = trips.drop(columns=['Inc_municipio', 'Inc_edad', 'Inc_act', 'Inc_fam', 'Inc_car'])
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,8944.175
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,8944.175
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,10539.365
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,25397.860
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27611.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21899,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,1,Hogar de una persona,1,6,3,1,2,2,2011,21474.460
21900,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,28070.860
21901,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,25703.710
21902,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,1,2 adultos,2,5,6,2,2,2,2011,13577.460


In [29]:
trips.to_csv("../input_data/trips.csv")

In [65]:
trips = pd.read_csv("../input_data/trips.csv",index_col=0)